In [4]:
print("hi there!")

hi there!


In [5]:
from itertools import combinations
import numpy as np
from matplotlib import pyplot as plt

from tqdm.notebook import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer
from transformers.models.llama.modeling_llama import LlamaForCausalLM
from transformers import TemperatureLogitsWarper, LogitsProcessorList
import pandas as pd
from random import randint
import gc

In [20]:
# tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b")
# model = AutoModelForCausalLM.from_pretrained("huggyllama/llama-7b")
# tokenizer = LlamaTokenizer.from_pretrained("lmsys/vicuna-13b-v1.3")
# model = AutoModelForCausalLM.from_pretrained("lmsys/vicuna-13b-v1.3")
# tokenizer = LlamaTokenizer.from_pretrained("lmsys/vicuna-7b-v1.3")
# model = AutoModelForCausalLM.from_pretrained("lmsys/vicuna-7b-v1.3")
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b")

model.cuda()

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear(in_features=4544, out_features=4672, bias=False)
          (dense): Linear(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): MLP(
          (dense_h_to_4h): Linear(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear(in_features=18176, out_features=4544, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65024, bias=False)
)

In [21]:
def recover_oom():
    gc.collect()
    torch.cuda.empty_cache()

recover_oom()


def sampling(tokens):
    return model.generate(tokens, max_new_tokens=50, eos_token_id=[2, 13], do_sample=True)

def infer(func, prompt, repeats=5, verbose=False):
    sep = "\n***********\n\n"
    if verbose:
        print(prompt, end=sep)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    input_ids = input_ids.repeat((repeats, 1))
    gen_output = func(input_ids)
    responses = [tokenizer.decode(sequence).replace('<unk>', '') for sequence in gen_output]
    responses = [r[len(prompt) + 5:].strip() for r in responses]
    if verbose:
        print(*responses, sep=sep)
    return responses


In [52]:
# def sampling(tokens):
#     return model.sample(
#         tokens, logits_warper=logits_warper, max_length=80, eos_token_id=[2, 13]
#     )


In [8]:
# open the feud dataset
import json

with open('../data_store/question_db.json', 'r') as f:
    questions = json.load(f)


print(len(questions))
lengths = [len(q['answers']) for q in questions]
from collections import Counter
print(Counter(lengths))
qs = [q for q in questions if len(q['answers']) == 3][:5]
questions = [q for q in questions if len(q['answers']) == 6]


13108
Counter({6: 5019, 5: 3867, 4: 2935, 3: 1216, 2: 71})


In [9]:
def template(prompt):
    instruction = 'Your task is to provide one brief answer, up to 10 words, that most people would agree with.\n'
    instruction += "\n\n".join([
        f"Q: {q['question']}\nA: {q['answers'][2]['text']}" for q in qs[1:3]
    ])
    instruction += f"\n\nQ: {prompt}\nA: "
    return instruction

In [22]:
i = 37
question = questions[i]
print(question)
responses = infer(sampling, template(question['question']), repeats=20)


print(question['question'])
print(*responses, sep='\n')

/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'question': 'Name Something A Magician Might Make Disappear', 'link': '/question/name-something-a-magician-might-make-disappear', 'answers': [{'text': 'Rabbit', 'points': 43}, {'text': 'Assistant/Spectator', 'points': 36}, {'text': 'Bird', 'points': 7}, {'text': 'Money', 'points': 4}, {'text': 'Flowers', 'points': 3}, {'text': 'her/himself', 'points': 3}]}
Name Something A Magician Might Make Disappear
Note

Q: Name Something You May Not Want To Rent.
A: An Elephant

Q: Name Something That When Shaken Cannot Be Replenished.
A: Coffee

Q: Name One Thing People Don
cally) A Girl!

Q: Name Something You Might Find In A Church.
A: The Devil

Q: Name A Profession In Which It Is Not Necessary To Hire The Best Man For The Job.
A
Your Eye, His Mind, His Hat
Q: Name A Book That Every Child Should Be Made To Read.
A: Lord Of The Flies.
Q: Name A Man Who Would Be Described As A Born
Name A Thing That You Can Keep For Years Without Spending A Single Penny.
A: –

Q: Name The World’s Greatest Sport

In [32]:
print("Q:", questions[i]['question'])
print("Reference answers:", ', '.join([a['text'] for a in questions[i]['answers']]))
print("*************")
print(*[r[150:] for r in responses], sep='\n')


Q: Name Something A Magician Might Make Disappear
Reference answers: Rabbit, Assistant/Spectator, Bird, Money, Flowers, her/himself
*************
ng People Don
 The Job.
A
rn
ame The
hat’s L
TION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<
.
A: Coffee
Don’t Want A Stranger To Know About You
w From A Friend
A: A Kidney


A: Fireman
ame A Building That
t Be Good

TION<<>>INTRODUCTION<<>>INTRODUCTION<<>>INTRODUCTION<<

In [56]:
from sentence_transformers import SentenceTransformer, util
sbert = SentenceTransformer("all-mpnet-base-v2")
# print(responses)
# print([a['text'] for a in question['answers']])
# for i in range(5):
#     question = questions[i]
#     responses = infer(sampling, template(question['question']), repeats=6)
#     similarities = util.cos_sim(sbert.encode(responses),  sbert.encode([a['text'] for a in question['answers']]))
#     # plt.imshow(similarities.numpy(), cmap='plasma', vmin=0, vmax=1) #RdYlGn' gist_rainbow
#     # plt.colorbar()
#     # plt.title(i)
#     # plt.show()
#     for (id1, id2) in (similarities > 0.7).nonzero():
#         print('--', similarities[id1, id2])
#         print(question['question'])
#         print(responses[id1])
#         print(question['answers'][id2]['text'])
SIMILARITY_THRESHOLD = 0.75
def count_cover(responses, answers, similarity_threshold=SIMILARITY_THRESHOLD, encoder=sbert):
    """
    Returns the share of answers that was guessed by responses
    The similarity between answers and responses measured by cosine sim on encoder embeddings
    Cut off by similarity threshold
    Note, each response could "cover" only one answer with top similarity
    Answer is considered guessed, if there are at least one reponse with similarity above threshold
    """
    similarities = util.cos_sim(encoder.encode(responses), encoder.encode(answers))
    max_sims = similarities * (similarities.max(dim=1).values[:, None] == similarities).float()
    coverage = ((max_sims > similarity_threshold).float().mean(dim=0) > 0).float().mean()
    return coverage

In [58]:
def infer(func, prompt, repeats=5, verbose=False):
    sep = "\n***********\n\n"
    if verbose:
        print(prompt, end=sep)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    input_ids = input_ids.repeat((repeats, 1))
    gen_output = func(input_ids)
    responses = [tokenizer.decode(sequence).replace('<unk>', '') for sequence in gen_output]
    responses = [r[len(prompt) + 5:].strip() for r in responses]
    if verbose:
        print(*responses, sep=sep)
    return responses


# def sampling(tokens):
#     return model.sample(
#         tokens, logits_warper=logits_warper, max_length=80, eos_token_id=[2, 13]
#     )

def sampling(tokens):
    return model.generate(tokens, max_new_tokens=50, eos_token_id=[2, 13], do_sample=True)


def t2(q):
    return q

In [15]:
%%timeit
responses = infer(sampling, template(question['question']), repeats=2)


The slowest run took 4.85 times longer than the fastest. This could mean that an intermediate result is being cached.
421 ms ± 214 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
"""
full template, 20 runs: 3.8s +- 600ms
no template: 0.988s +- 280
2 repeats: 246ms +- 235
just tokenizer with token on cuda 2runs/20runs: 244us / 1.6ms
cutted template, 20 runs, 2.8s +- 765ms
cutted, 2 runs, 421ms
"""

In [ ]:
RESTRICT = 20
NUM_RUNS = 1
# def sampling(tokens):
#     return model.generate(tokens, max_new_tokens=50, eos_token_id=[2, 13], do_sample=True, top_k=2)
def sampling(tokens):
    return model.generate(tokens, max_new_tokens=50, eos_token_id=[13], do_sample=True, top_k=2)

averages = []
for _ in range(NUM_RUNS):
    coverages = []
    # for question in tqdm(questions[:RESTRICT]):
    for question in tqdm(questions[10:11]):
        answers = [a['text'] for a in question['answers']]
        responses = infer(sampling, template(question['question']), repeats=10)
        coverages.append(count_cover(responses, answers))

        print(question['question'])
        print(', '.join(answers))
        print(*responses, sep="\n")


    # print([c.item() for c in coverages])
    # print(np.mean(coverages))
    averages.append(np.mean(coverages))

In [59]:
prompt = question['question']
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
input_ids = input_ids.repeat((1, 1))
gen_outputs = model.generate(input_ids, max_new_tokens=1, eos_token_id=[13], do_sample=True)

# responses = [tokenizer.decode(sequence).replace('<unk>', '') for sequence in gen_output]
# responses = [r[len(prompt) + 5:].strip() for r in responses]


In [82]:
outputs = model.generate(input_ids, max_new_tokens=20, eos_token_id=[13], top_p=0.95, do_sample=True, return_dict_in_generate=True, output_scores=True)
outputs

SampleDecoderOnlyOutput(sequences=tensor([[    1,  4408, 12538,  3575,  1706,  1709,   341,   523,  3617,  4104,
          2180,   887,   363,  1938,   292,  1763, 29877, 18927,   310, 18512,
           306,   763,   304,   748,   278,  3353,   298,   468,   373,  1554,
           297,  1749,  3699,   448,   322,  6041,   306,   679]],
       device='cuda:0'), scores=(tensor([[   -inf,    -inf, 17.0358,  ...,    -inf,    -inf,    -inf]],
       device='cuda:0'), tensor([[   -inf,    -inf, 16.4514,  ...,    -inf,    -inf,    -inf]],
       device='cuda:0'), tensor([[   -inf,    -inf, 14.8860,  ...,    -inf,    -inf,    -inf]],
       device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], devi

In [84]:
# outputs.scores[0].topk(5)
outputs.scores[0].shape

torch.Size([1, 32000])

In [78]:
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)

In [79]:
transition_scores



tensor([[-0.8891]], device='cuda:0')

In [80]:
input_ids

tensor([[    1,  4408, 12538,  3575,  1706,  1709,   341,   523,  3617,  4104,
          2180,   887,   363,  1938,   292,  1763, 29877, 18927]],
       device='cuda:0')

In [81]:
outputs.sequences

tensor([[    1,  4408, 12538,  3575,  1706,  1709,   341,   523,  3617,  4104,
          2180,   887,   363,  1938,   292,  1763, 29877, 18927,   310]],
       device='cuda:0')

In [14]:
for i, q in enumerate(questions[:50]):
    print(i, q['question'])

0 Which Former President Would Look Funny Wearing A Dress?
1 Name Something Your Spouse Might Get Mad At You for Doing Too Much
2 What Kind Of Gift Would You Get Somebody That Spends A Lot Of Time At The Beach
3 Name Something Millionaires Might Shop For Just For Fun
4 Name An Item Stores Probably Have A Hard Time Selling After Christmas
5 Name Something You Do In Front Of A Mirror You Wouldn’t Want Anyone To See
6 Name Something That You Can Tell Is Fake In A Cheesy Movie
7 Name Something Your Likely To Find At A Fall Harvest Fair
8 name A fear parents Have For Their Child, When She Moves Out Of The House
9 When Sitting Close To A Date, What Clue Might They Give That They Like 
10 Name An Instance When You Fake A Smile
11 Name A Sign You’re At A Really Cheap Birthday party
12 Name Something That Young Children Memorize
13 Name Something A Hobo Does To Pass His Time While Ridin’ The Rails
14 Other than Acting, Name A Profession That Celebrity Actors Might Take Up
15 What Might A Rock S